In [1]:
import pandas as pd
import datasets

/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/taken_for_human_data/filtered_samples.csv')



In [3]:
df

,paper_id,venue,focused_review,point,Discard
0,ACL_2017_270_review,ACL_2017,"Unfortunately, there are not many new ideas in...",2) the effect of removing inference compositio...,1.0
1,ACL_2017_588_review,ACL_2017,and the evaluation leaves some questions unans...,1) An important assumption being made is that ...,NaN
2,ACL_2017_108_review,ACL_2017,The problem itself is not really well motivate...,- In GENIA dataset: On the GENIA dataset - out...,NaN
3,ACL_2017_818_review,ACL_2017,1) Many aspects of the approach need to be cla...,"248 ""Above definition"": determiner missing Sec...",NaN
4,ACL_2017_606_review,ACL_2017,- [Choice of Dataset] The authors use WebQuest...,- [Choice of Dataset] The authors use WebQuest...,NaN
...,...,...,...,...,...
1195,tj4a1JY03u,ICLR_2024,1. The conclusions are a bit obvious - that hi...,"3. The evaluation is limited, mostly relying o...",NaN
1196,8Ezv4kDDee,ICLR_2025,- Limited Practical Impact of Theoretical Form...,- Limited Practical Impact of Theoretical Form...,NaN
1197,S8VFVe6MWL,ICLR_2025,"Overall, the authors tend to trust subjective ...",3. The ablations seem to deserve better experi...,NaN
1198,t1nZzR7ico,ICLR_2025,1. The presentation in the experiment section ...,3.1. VioT dataset:20 images in each of the 4 c...,NaN


In [4]:
## reomvoe rows that has the value 1 in column E
df = df[df['Discard'] != 1.0]

In [5]:
len(df)

1082

In [7]:
## shuffle the rows
df = df.sample(frac=1).reset_index(drop=True)

# remove column 'Discard'
df = df.drop(columns=['Discard'])
df

,paper_id,venue,focused_review,point
0,ACL_2017_433_review,ACL_2017,- The annotation quality seems to be rather po...,- It would be helpful if you provided glosses ...
1,ACL_2017_67_review,ACL_2017,The main weaknesses for me are evaluation and ...,- Memb is apparently the previous state-of-the...
2,ICLR_2023_1833,ICLR_2023,. Strengths first:\nThe paper is one of the fi...,2) why not to consider finer grouping for quan...
3,NIPS_2022_1402,NIPS_2022,1. The representation could be further improve...,2. It would be better to study the impact of t...
4,ACL_2017_371_review,ACL_2017,- The description is hard to follow. Proof-rea...,- Why are you using GRU for the Pyramid and LS...
...,...,...,...,...
1077,NIPS_2020_1606,NIPS_2020,- All theoretical contributions are based on a...,1. The proposed model over-siplifies he proble...
1078,MbKRJUowYX,EMNLP_2023,1.In the introduction and the experimental ana...,1.In the introduction and the experimental ana...
1079,NIPS_2019_1130,NIPS_2019,weakness is the lack of focus of the discussio...,4. Line 277-286: This is an interesting observ...
1080,HzecOxOGAS,EMNLP_2023,"- A few parts lack details. E.g., Line 220, we...","- A few parts lack details. E.g., Line 220, we..."


In [11]:
## split the data into batches of 200 samples each, and convert them into hf datasets
batch_size = 200
batches = [df[i:i+batch_size] for i in range(0, len(df), batch_size)]
## concat the last two batches
batches[-2] = pd.concat([batches[-2], batches[-1]])
batches.pop()
len(batches)




200
200
200
200
282


In [12]:
## load HF authentication token from .env file
import os
from dotenv import load_dotenv
load_dotenv()
hf_token = os.getenv("HF_TOKEN")


## push the datasets to HF hub
for i,b in enumerate(batches):
    batch = i + 6
    b = datasets.Dataset.from_pandas(b)
    
    b.push_to_hub(f"boda/review_evaluation_main_{batch}",token=hf_token)



Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.51s/it]
